In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Install Simple Transformers library

In [20]:
# install simpletransformers
!pip install simpletransformers

# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

simpletransformers==0.60.6


In [21]:
!pip install tokenizers==0.9.4

# Load the dataset

In [22]:
import pandas as pd

df = pd.read_csv('/content/gdrive/MyDrive/stemaway/Codeacademy_Webscrapper_20210107154307.csv')

print(df.shape)
df.head()

(5464, 8)


,Unnamed: 0,Topic Title,Category,Tags,Leading Comment,Other Comments,Likes,Views
0,0,[How to] Format code in posts,Getting Started,[],Codecademy Discuss posts are parsed with Markd...,[],21,13.9k
1,1,Guide: A list of how-to’s for giving and recei...,Getting Started,[],Technical guides will teach you how to use the...,[],0,0
2,2,Guide: Learn about our community forums,Getting Started,[],Take a moment and read through each of these b...,[],0,0
3,3,[How-to] Create a topic that everyone will read,Getting Started,[],A great topic is one that provides potential h...,[],0,0
4,4,[Learn More] How the forum is organized,Getting Started,[],"With Codecademy Community forums, conversation...",[],0,0


In [23]:
df['Category'].unique()

array(['Getting Started', 'Get Help', 'Community', 'Projects', 'FAQ'],
      dtype=object)

In [24]:
class_list = ['Getting Started', 'Get Help', 'Community',
       'Projects', 'FAQ']
new_df = pd.DataFrame()
new_df['post'] = df['Topic Title'] + ' ' + df['Tags'] + ' ' + df['Leading Comment'] + ' ' + df['Other Comments']
new_df['category'] = df['Category']
new_df['post'] = new_df['post'].apply(lambda x: str(x))
new_df['category'] = new_df['category'].apply(lambda x: str(x))

In [25]:
# changing class names (text) to numbers (a necessity for simpletransformers library)
new_df['category'] = new_df.apply(lambda x:  class_list.index(x['category']),axis=1)

In [26]:
new_df.head()

,post,category
0,[How to] Format code in posts [] Codecademy Di...,0
1,Guide: A list of how-to’s for giving and recei...,0
2,Guide: Learn about our community forums [] Tak...,0
3,[How-to] Create a topic that everyone will rea...,0
4,[Learn More] How the forum is organized [] Wit...,0


In [27]:
new_df['category'].unique()

array([0, 1, 2, 3, 4])

In [28]:
# Split the data into train and test
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(new_df, test_size=0.10)

print('train shape: ',train_df.shape)
print('test shape: ',test_df.shape)

train shape:  (4917, 2)
test shape:  (547, 2)


# Load pre-trained model

# **Bert**

In [29]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-cased",
    num_labels=12,
    args=train_args
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

# Train model

In [30]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:446: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4917 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

(2460, 0.22533708242143455)

In [31]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result_Bert, model_outputs, wrong_predictions = model.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/547 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/69 [00:00<?, ?it/s]

In [32]:
result_Bert

{'acc': 0.9542961608775137,
 'eval_loss': 0.2636636665543926,
 'f1': 0.9542961608775137,
 'mcc': 0.9356908763947899}

# save and load the model

In [33]:
import os
import tarfile

def save_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [34]:
save_model('outputs','codeacademy-bert-categclass')

In [35]:
!tar -zxvf ./codeacademy-bert-categclass.tar.gz

outputs/eval_results.txt
outputs/pytorch_model.bin
outputs/special_tokens_map.json
outputs/model_args.json
outputs/tokenizer_config.json
outputs/training_args.bin
outputs/config.json
outputs/vocab.txt


In [36]:
!rm -rf outputs

# Test the loaded model on a real example

In [37]:
import os
import tarfile

def unpack_model(model_name=''): 
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

unpack_model('codeacademy-bert-categclass')

In [38]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "outputs/",
    num_labels=12,
    args=train_args
)

In [39]:
class_list = ['Getting Started', 'Get Help', 'Community',
       'Projects', 'FAQ']

post1 = "Guide: Learn about our community forums"

predictions, raw_outputs = model.predict([post1])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Getting Started


In [40]:
post2 = "About the Get Help category"

predictions, raw_outputs = model.predict([post2])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Get Help


In [44]:
post3 = "How to start working in my website"

predictions, raw_outputs = model.predict([post3])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Getting Started


In [42]:
post4 = "Python Syntax: Medical Insurance Project"

predictions, raw_outputs = model.predict([post4])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Projects


In [43]:
post5 = "Why is my Triangle class not accepted?"

predictions, raw_outputs = model.predict([post5])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Community


# **Roberta**

In [45]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model_rob = ClassificationModel(
    'roberta', 'roberta-base',
    num_labels=7,
    args=train_args
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

# Train model

In [75]:
# Train the model
model_rob.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:446: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4917 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

(2460, 0.22337470463593254)

In [76]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result_Roberta, model_outputs, wrong_predictions = model_rob.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/547 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/69 [00:00<?, ?it/s]

In [77]:
result_Roberta

{'acc': 0.9561243144424132,
 'eval_loss': 0.25624893346389727,
 'f1': 0.9561243144424132,
 'mcc': 0.9382386217902128}

# save and load the model

In [82]:
import os
import tarfile

def save_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [79]:
save_model('outputs','codeacademy-roberta-categclass')

In [80]:
!tar -zxvf ./codeacademy-roberta-categclass.tar.gz

outputs/vocab.json
outputs/eval_results.txt
outputs/pytorch_model.bin
outputs/special_tokens_map.json
outputs/model_args.json
outputs/tokenizer_config.json
outputs/merges.txt
outputs/training_args.bin
outputs/config.json
outputs/vocab.txt


In [81]:
!rm -rf outputs

# Test the loaded model on a real example

In [83]:
import os
import tarfile

def unpack_model(model_name=''): 
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

unpack_model('codeacademy-roberta-categclass')

In [85]:
class_list = ['Getting Started', 'Get Help', 'Community',
       'Projects', 'FAQ']

post6 = "[How-to] Create a topic that everyone will read"

predictions, raw_outputs = model.predict([post6])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Getting Started


In [86]:
post7 = "Code won’t run and it’s not offering to show a solution"

predictions, raw_outputs = model.predict([post7])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Community


In [87]:
post8 = "Codeacademy down?"

predictions, raw_outputs = model.predict([post8])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Community


In [88]:
post9 = "Prime Directive project - solution"

predictions, raw_outputs = model.predict([post9])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Projects


In [89]:
post10 = "FAQ: Learn Python - Battleship - Not Again!"

predictions, raw_outputs = model.predict([post10])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

FAQ


# DistilBert

In [90]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}

# Create a ClassificationModel
model_distil = ClassificationModel(
    'distilbert', 'distilbert-base-cased',
    num_labels=7,
    args=train_args
)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

# Train model

In [91]:
# Train the model
model_distil.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:446: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4917 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

(2460, 0.22202784868057324)

In [92]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result_distilbert, model_outputs, wrong_predictions = model_distil.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/547 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/69 [00:00<?, ?it/s]

In [93]:
result_distilbert

{'acc': 0.926873857404022,
 'eval_loss': 0.3130772908771178,
 'f1': 0.926873857404022,
 'mcc': 0.8969175727810633}

# save and load the model

In [94]:
import os
import tarfile

def save_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [95]:
save_model('outputs','codeacademy-distilbert-categclass')

In [96]:
!tar -zxvf ./codeacademy-distilbert-categclass.tar.gz

outputs/vocab.json
outputs/eval_results.txt
outputs/pytorch_model.bin
outputs/special_tokens_map.json
outputs/model_args.json
outputs/tokenizer_config.json
outputs/merges.txt
outputs/training_args.bin
outputs/config.json
outputs/vocab.txt


In [97]:
!rm -rf outputs

# Test the loaded model on a real example

In [98]:
import os
import tarfile

def unpack_model(model_name=''): 
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

unpack_model('codeacademy-distilbert-categclass')

In [99]:
class_list = ['Getting Started', 'Get Help', 'Community',
       'Projects', 'FAQ']

post11 = "[Learn more] Welcome to the Codecademy Community Forums!"

predictions, raw_outputs = model.predict([post11])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Getting Started


In [100]:
post12 = "Chore Door innerHTML error"

predictions, raw_outputs = model.predict([post12])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Get Help


In [101]:
post13 = "Re-creating the lodash library - .pad() need help"

predictions, raw_outputs = model.predict([post13])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Projects


In [102]:
post14 = "I hit the 90 day mark!"

predictions, raw_outputs = model.predict([post14])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Community


In [103]:
post15 = "Why does it ask me not to use compute_bill?"

predictions, raw_outputs = model.predict([post15])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Community


# Xlnet

In [104]:
from simpletransformers.classification import ClassificationModel

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 4}


# Create a ClassificationModel
model_xlnet = ClassificationModel(
    'xlnet', 'xlnet-base-cased',
    num_labels=7,
    args=train_args
)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

# Train model

In [105]:
# Train the model
model_xlnet.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:446: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/4917 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/615 [00:00<?, ?it/s]

(2460, 0.22031427620905908)

In [106]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result_xlnet, model_outputs, wrong_predictions = model_xlnet.eval_model(test_df, f1=f1_multiclass, acc=accuracy_score)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:1026: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/547 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/69 [00:00<?, ?it/s]

In [107]:
result_xlnet

{'acc': 0.9451553930530164,
 'eval_loss': 0.2609533626794249,
 'f1': 0.9451553930530164,
 'mcc': 0.9226777000597491}

# save and load the model

In [108]:
import os
import tarfile

def save_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

In [109]:
save_model('outputs','codeacademy-xlnet-categclass')

In [110]:
!tar -zxvf ./codeacademy-xlnet-categclass.tar.gz

outputs/vocab.json
outputs/eval_results.txt
outputs/pytorch_model.bin
outputs/special_tokens_map.json
outputs/model_args.json
outputs/tokenizer_config.json
outputs/merges.txt
outputs/training_args.bin
outputs/config.json
outputs/vocab.txt
outputs/spiece.model


In [113]:
!rm -rf outputs

# Test the loaded model on a real example

In [114]:
import os
import tarfile

def unpack_model(model_name=''): 
  tar = tarfile.open(f"{model_name}.tar.gz", "r:gz")
  tar.extractall()
  tar.close()

unpack_model('codeacademy-xlnet-categclass')

In [115]:
class_list = ['Getting Started', 'Get Help', 'Community',
       'Projects', 'FAQ']

post16 = "[Learn more] Welcome to the Codecademy Community Forums!"

predictions, raw_outputs = model.predict([post16])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Getting Started


In [116]:
post17 = "Chore Door innerHTML error"

predictions, raw_outputs = model.predict([post17])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Get Help


In [117]:
post18 = "Re-creating the lodash library - .pad() need help"

predictions, raw_outputs = model.predict([post18])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Projects


In [118]:
post19 = "I hit the 90 day mark!"

predictions, raw_outputs = model.predict([post19])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Community


In [119]:
post20 = "Why does it ask me not to use compute_bill?"

predictions, raw_outputs = model.predict([post20])

print(class_list[predictions[0]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Community


# Result

In [122]:
import pandas as pd
result=pd.DataFrame({'Model_Name': ['BERT', 'roberta', 'distilbert','xlnet'],
                     'Accuracy':[result_Bert['acc'],result_Roberta['acc'],result_distilbert['acc'],result_xlnet['acc']],
                     'Evaluation_loss':[result_Bert['eval_loss'],result_Roberta['eval_loss'],result_distilbert['eval_loss'],result_xlnet['eval_loss']],
                      'F1_Score':[result_Bert['f1'],result_Roberta['f1'],result_distilbert['f1'],result_xlnet['f1']],
                      'MCC':[result_Bert['mcc'],result_Roberta['mcc'],result_distilbert['mcc'],result_xlnet['mcc']]})

In [123]:
result

,Model_Name,Accuracy,Evaluation_loss,F1_Score,MCC
0,BERT,0.954296,0.263664,0.954296,0.935691
1,roberta,0.956124,0.256249,0.956124,0.938239
2,distilbert,0.926874,0.313077,0.926874,0.896918
3,xlnet,0.945155,0.260953,0.945155,0.922678
